In [1]:
from datetime import datetime, timedelta


class Bucket:
    def __init__(self,period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0 # this is incorrect an not dynamic on #use property instead to get dynamic attribtes
    def __repr__(self):
        return f"Bucket (quota={self.quota})"

In [2]:
#fill will add amount to quoto after period of time
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

In [3]:
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False                # Bucket hasn't been filled this period.
    if bucket.quota - amount < 0:
        return False                # Bucket was filled, but not enough
    bucket.quota -= amount 
    return True                     # Bucket has enough, quota consumed

In [4]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket (quota=100)


In [5]:
if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print(bucket)

Not enough for 99 quota
Bucket (quota=100)


In [6]:
if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print(bucket)

Not enough for 3 quota
Bucket (quota=100)


In [9]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    def __repr__(self):
        return (f"NewBucket(max_quota={self.max_quota}, "
                f"quota_consumed={self.quota_consumed})")
    
    @property #property decorator will turn method into attribute
    def quota(self):
        return self.max_quota - self.quota_consumed #this is the correct definition about quota

    @quota.setter #qquota.setter is call when newbucket.quota assign new value
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0: #case for reset operation
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0: #case for fill operation
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else: #case for deduct operation
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [10]:
bucket = NewBucket(60)
print("Initial", bucket)
fill(bucket, 100)
print("Filled", bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Not enough for 99 quota")

print("Now", bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print("Still", bucket)

Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now NewBucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still NewBucket(max_quota=100, quota_consumed=99)


**Note about NewBucket:**
- we use @property decorator to convert a function in to attribute.
- to get the value of new property or attribute use like normal attribute. For example, new_bucket.quota.
- use decorator @property_name.setter to turn method into property setter method
- by define quota using @property and @quota.setter we can reuse fill and deduct method.
- @quota.setter method execute when self.quota have new assignment operation

**Note about fill and deduct method:**
- fill(self, amount):  update self.quota += amount
- deduct(self, amount): update filling perios status which return True or False. If True update self.quota -= amount

**Summary:**
- use @property decorator to turn a method into an attribute
- use @property_name to define property setter method. @property_name.setter to turn method into property setter method